# Lab 02 - Wprowadzenie do AI

Celem tego notebook'a jest przejście przez podstawowe zagadnienia związane z uczeniem maszynowym. Wszystko można wykonać u siebie lokalnie instalując biblioteki z pliku `requirements.txt`, natomiast można wszystko również wykonać przy wykorzystaniu Google Colab

[![Otwórz w Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/aghbit/BIT-AI-Intro/blob/main/lab_02/main.ipynb)

In [ ]:
import optuna
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

random.seed(2)
np.random.seed(2)

---
## Szukanie optymalnego rozwiązania funkcji

Zanim przejdziemy do tematyki związanej z uczeniem maszynowym, zajmiemy się zagadnieniem optymalizacji funkcji. Dzisiaj zajmiemy się funkcją

$$f(x) = (x + 0.1)^2-\frac{7x}{10}\sin(20 * x) + 1$$

Oczywiście jeżeli poświęcimy wystarczająco dużo czasu, to możemy rozwiązać to zadanie analitycznie, natomiast to nie jest celem tego ćwiczenia.

Na początku zwizualizujemy funkcję, aby zobaczyć jak wygląda. W tym celu wykorzystamy bibliotekę `matplotlib` oraz `numpy`.

In [ ]:
def func(x):
    return (x + 0.1) ** 2 - 7 * x * np.sin(20 * x) / 10 + 1 

def create_plot(guesses=None):
    x = np.linspace(-1.5, 1.5, 1000)
    y = func(x)
    plt.figure(figsize=(10, 6))
    plt.plot(x, y, label="f(x)")

    if guesses is not None:
        print("Attemps: ", guesses)
        y0 = func(guesses)
        plt.scatter(guesses, y0, color='red', label="Guesses", s=100, marker='x')

    plt.title("Objective Function")
    plt.xlabel("x")
    plt.ylabel("y")
    plt.legend()
    plt.grid()
    plt.show()

create_plot(np.array([0, 1]))


---
### Random Search

Najprostszą metodą na znalezienie dobrego rozwiązania jest przeszukanie losowe naszej dziedziny. Random Search polega na losowaniu punktów i sprawdzaniu ich wartości. Oczywiście zazwyczaj obliczenie wartości funkcji jest kosztowne, więc nie możemy sobie pozwolić na zbyt wiele prób. Tutaj, ograniczymy się do 10 prób.

**Zadanie 1.**
Zaimplementować funkcję `suggest_random_x`, która będzie losować punkty w zakresie $[a, b]$

In [ ]:
def suggest_random_x(a, b):
    raise NotImplementedError("This function should be implemented by the user.")

guesses = np.array([suggest_random_x(-1.5, 1.5) for _ in range(10)])
create_plot(guesses)

In [ ]:
print("Top Scores: ", min(func(guesses)))

---
### Optuna

Może i wynik nie jest najgorszy, natomiast nie jest to najlepsza metoda. Często okazuje się, że informację o wartościach funkcji w innych miejscach możemy wykorzystać w następnym zgadywaniu. Jedną z bibliotek, która to robi jest `Optuna`. Aby z niej skorzystać, musimy zdefiniować funkcję celu, która będzie zgadywała wartości funkcji i zwracała jej wartość. Przykładowa funkcja celu wygląda tak:

```python
def objective(trial):
    x = trial.suggest_int("x", 0, 10)
    return (x - 4) ** 2
```

W powyższym przykładzie, `trial` to obiekt, który przechowuje informacje o próbie. `suggest_float` to metoda, która losuje wartość zmiennej `x` z przedziału [0, 10]. Następnie zwracamy wartość funkcji celu. Wartością zwracaną przez funkcję celu jest wartość funkcji, którą chcemy zminimalizować. Przykład wykorzystania znajdziecie w tym [tutorialu](https://optuna.readthedocs.io/en/stable/tutorial/20_recipes/003_attributes.html).

**Zadanie 2.**
Zaimplementować funckję celu wykorzystując bibliotekę `Optuna`.

In [ ]:
def objective(trial):
    raise NotImplementedError("This function should be implemented by the user.")

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=10)
print("Best Score: ", study.best_value)
print("Best Params: ", study.best_params)
all_guesses = [trial.params["x"] for trial in study.trials]
create_plot(np.array(all_guesses))

**Zadanie 3.**
Poniżej znajduje się funkcja Rastrigin, która jest funkcją testową. Wykorzystać `Random Search` oraz `Optuna` do znalezienia minimum tej funkcji. Funkcja Rastrigin jest zdefiniowana jako:
$$f(x) = An + \sum_{i=1}^{n} [x_i^2 - A \cos(2 \pi x_i)]$$

Która poradziła sobie lepiej? Porównaj wyniki dla 2, 5 i 100 wymiarów.

In [ ]:
def rastrigin(x, A=10):
    return A * len(x) + sum([(xi ** 2 - A * np.cos(2 * np.pi * xi)) for xi in x])

raise NotImplementedError("This code should be implemented by the user.")

---
## Hiperparametry

Modele uczenia maszynowego często mają hiperparametry. Modele przy poprawnie zaimplementowanej metodzie uczenia, dla stałych danych, za każdym razem zwrócą ten sam wynik. Natomiast zmieniając hiperparametry, możemy uzyskać różne wyniki.

Naszym celem będzie teraz wykorzystanie wiedzy nabytej wcześniej, aby poprawić wynik modelu.

Tym razem będziemy musieli napisać trochę kodu sami (natomiast możecie wracać do kodu z wcześniejszych zajęć, który uzupełniliście sami w wolnym czasie, zrobiliście to, prawda?). Wracamy do danych z ostatniego labu, czyli do danych o `titanicu`. Ładujemy plik `data/titanic.csv` za pomocą bilioteki `pandas`. Dokumentacja:

### pandas
- [read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html)
- [mean](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.mean.html)
- [fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html)

### scikit-learn
- [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

**Zadanie 4.**
1. Załaduj dane
2. Zostaw kolumny: `Survived`, `Pclass`, `Age`, `SibSp`, `Parch`, `Fare`
3. Uzupełnij braki średnimi wartościami
4. Podziel dane na zbiór treningowy i testowy (70% - 30%)

In [ ]:
raise NotImplementedError("This code should be implemented by the user.")

Następnie wykorzystamy model `RandomForestClassifier` z biblioteki `scikit-learn`, do uzyskania początkowego modelu. Przetestujemy jego dokładność na zbiorze testowym, co będzie naszym punktem wyjścia.

**Zadanie 5.**
1. Załaduj model `RandomForestClassifier` z biblioteki `scikit-learn` (ustaw random_state na 42)
2. Wytrenuj model na zbiorze treningowym
3. Sprawdź dokładność modelu na zbiorze testowym przy wykorzystaniu metryki `accuracy`

In [ ]:
raise NotImplementedError("This code should be implemented by the user.")

Teraz możemy przejśc do optymalizacji hiperparametrów. Dokumentację tej klasy możemy znaleźć [tutaj](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html). 

**Zadanie 6.**
Wykorzystaj bibliotekę `Optuna` do optymalizacji hiperparametrów modelu `RandomForestClassifier`.

In [ ]:
raise NotImplementedError("This code should be implemented by the user.")

---
## Bayes error rate

Istnieje twierdzenie, które mówi nam o najniższym możliwym błędzie klasyfikacji, który możemy uzyskać. Jest to tzw. Bayes error rate ([link dla zainteresowanych](https://en.wikipedia.org/wiki/Bayes_error_rate])). W dużym skrócie, czasami lepiej się nie da i trenowanie modelu nie pomoże.

W uproszczeniu. Mamy samochody marki `X`. Każdy samochód marki `X` ma wagę i moc silnika (i to są jedyne parametry tego samochodu, załóżmy że reszta znajduje się w stanie kwantowym i nic o nich nie wiemy), a my chcemy stworzyć model, który na podstawie wagi, będzie zwracał moc silnika. Co więcej, wiemy, że moc silnika jest opisana jako:

$$\frac{\texttt{waga}}{10}\pm20$$

matematycznie zapiszemy to jako:
$$\texttt{moc} = \frac{\texttt{waga}}{10} + U(-20, 20)$$

Czyli auto, które waży 2 tony, będzie miało moc pomiędzy 180 a 220 KM. W takim przypadku, nawet jeżeli stworzymy najlepszy model na świecie, to i tak nie będziemy w stanie przewidzieć mocy silnika z wagą. 

Zależnie od metryki, którą zastosujemy, możemy dostać trochę inne modele, które będą optymalne dla tak opisanych samochodów marki `X`.

Jeżeli zastosujemy metryki `MSE`, `MAE`, `RMSE`, to optymalny model będzie opisany jako:
$$\frac{\texttt{waga}}{10}$$

Jeżeli zastosujemy metrykę `Coverage(k)`, to optymalnymi modelami będą wszystkie modele opisane jako:
$$\frac{\texttt{waga}}{10} + b$$
gdzie $b$ to dowolna liczba z przedziału $[-(20-k), 20-k]$.
